In [1]:
import openpyxl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import operator

In [2]:
class DataEntry:
    def __init__(
        self,
        phase,
        category,
        name,
        description,
        units,
        paramType,
        param1,
        param2,
        param3,
        param4,
        param5,
        param6,
        param7,
        lowerLimit,
        upperLimit,
        step,
        applicationMethod,
        surfaceType,
        *args, 
        **kwargs):
        self.Phase = phase
        self.Category = category
        self.Description = description
        self.Name = name
        self.Units = units
        self.ParameterType = paramType
        self.Param1 = param1 if param1 is not None else 0.0
        self.Param2 = param2 if param2 is not None else 0.0
        self.Param3 = param3 if param3 is not None else 0.0
        self.Param4 = param4 if param4 is not None else 0.0
        self.Param5 = param5 if param5 is not None else 0.0
        self.Param6 = param6 if param6 is not None else 0.0
        self.Param7 = param7 if param7 is not None else ''
        self.LowerLimit = lowerLimit
        self.UpperLimit = upperLimit
        self.Step = step
        self.ApplicationMethod = applicationMethod
        self.SurfaceType = surfaceType

    def __str__(self):
        return f'Name = {self.Name};Type = {self.ParameterType}'
    
    @staticmethod
    def FromExcel(row):
        return DataEntry(
            *(r.value for r in row)
        )

In [3]:
wb = openpyxl.load_workbook('ModifyParameters.xlsx')

In [8]:

deconSheets = [
    'Fumigation',
    'Physical',
    'Liquid Spray',
    'Liquid Immersion',
    'Foam Spray',
    'Gel',
    'Aerosol',
    'Liquid Wipe',
    'Fogging',
    'Liquid Suspension'
]
deconSheets = sorted(deconSheets)
deconSheets

['Aerosol',
 'Foam Spray',
 'Fogging',
 'Fumigation',
 'Gel',
 'Liquid Immersion',
 'Liquid Spray',
 'Liquid Suspension',
 'Liquid Wipe',
 'Physical']

In [5]:
for sheet_name in deconSheets:
    print(sheet_name)
    sheet = wb.get_sheet_by_name(sheet_name)
    data_cells = [[sheet.cell(r,c) for c in range(1, sheet.max_column + 1)] for r in range(2, sheet.max_row + 1)]
    entries = [DataEntry.FromExcel(r) for r in data_cells]
    uniform_xd = [e for e in entries if e.ParameterType == 'UniformXDependent']

    grouped = {}
    for e in uniform_xd:
        if e.SurfaceType + '-' + e.Param7 not in grouped:
            grouped[e.SurfaceType + '-' + e.Param7] = []
        grouped[e.SurfaceType + '-' + e.Param7].append(e)

    if not os.path.exists(sheet_name):
        os.makedirs(sheet_name)
    for key in list(grouped.keys()):
        starting = {(g.Param3, g.Param4) for g in grouped[key]}
        ending = {(g.Param5, g.Param6) for g in grouped[key]}
        points = list(starting|ending)
        points = sorted(points, key=operator.itemgetter(0,1))
        points = [list(p) for p in points]
        f = open(f'{sheet_name}\\{key}.csv', 'w')
        f.write('x,y\n')
        for p  in points:
            x,y = tuple(p)
            f.write(f'{x},{y}\n')
        f.close()

Fumigation
Physical
Liquid Spray
Liquid Immersion
Foam Spray
Gel
Aerosol
Liquid Wipe
Fogging
Liquid Suspension
